In [ ]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import getpass
import time
import pandas as pd

Portfolio Data Collection and Display

In [ ]:
##LOADING USER PORTFOLIO DATA

from selenium.webdriver import Chrome
driver = webdriver.Chrome("C:/Projects/chromedriver_win32/chromedriver.exe")
driver.get('https://investor.dsetrade.com/Cstfwsrv2012/html/MainHome.aspx')

#sending BO ID for the first login page
first_login = driver.find_element_by_id("TextBoxLogin")
driver.implicitly_wait(7)
username = input("Please type your username used to access your portfolio: ")
first_login.send_keys(username)
driver.implicitly_wait(3)
driver.find_element_by_class_name("button-gris1").click()

#sending password to second login page
driver.implicitly_wait(7)
second_login = driver.find_element_by_id("_xcologin_pwd")
password = getpass.getpass("Please enter your password ")
second_login.send_keys(password)
driver.find_element_by_id("_xcologin_connect").click()

#inputting 2FA code from mobile
driver.implicitly_wait(7)
two_fa = input("Enter the 2FA code from your mobile ")
third_login = driver.find_element_by_id("_xcologin_otp")
third_login.send_keys(two_fa)
driver.implicitly_wait(7)
driver.find_element_by_id("_xcologin_confirm_otp").click()

#extracting portfolio information
time.sleep(3)
portfolio_a = []
portfolio_b = []

#fetching "nlong" class elements
fetch_portfolio_a = driver.find_elements_by_css_selector("div.nlong")
for i in fetch_portfolio_a:
    if (i.text !=''):
        portfolio_a.append(i.text)

#fetching "nlong_char" class elements
fetch_portfolio_b = driver.find_elements_by_css_selector("div.nlong_9char")
for i in fetch_portfolio_b:
    if (i.text !=''):
        portfolio_b.append(i.text)
        

#joining them together
full_portfolio = portfolio_a + portfolio_b 

#fetching position in each company for later use 
empty = []
empty_2 = []
c = driver.find_elements_by_class_name("data_A2")
for _ in c:
        empty.append(_.text)
for i in empty:
    try:
        if isinstance(int(i),int):
            empty_2.append(i)
    except ValueError:
        pass
p_data = (empty_2[1::2])
p_data = list(map(str,p_data))
weight_data = dict(zip(full_portfolio,p_data))
df_1 = pd.DataFrame(weight_data,index = ["Position(BDT)"])
driver.quit()
print("\n Portfolio Tickers have been collected. Moving on to data collection")

#RATIO COLLECTION
data = {}
print("Starting Data Collection. Please Wait!")
for company in full_portfolio:
    driver = webdriver.Chrome("C:/Projects/chromedriver_win32/chromedriver.exe")
    driver.get('https://www.amarstock.com/stock/%s'% company)
    time.sleep(1)
    pe_ratio = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[1]/div/div[3]") 
    beta = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[5]/div/div[3]")
    eps = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[2]/div/div[3]")
    price_to_book = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[3]/div/div[3]")
    div_yield = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[6]/div/div[3]")
    ratios = [pe_ratio.text,beta.text,eps.text,price_to_book.text,div_yield.text]
    data[company] = ratios
    driver.quit()
df = pd.DataFrame(data, index = ["P/E Ratio","Beta","EPS","Price-to-book","Dividend Yield",])
portfolio_data = df.append(df_1)
for company in full_portfolio:
    portfolio_data[company] = pd.to_numeric(portfolio_data[company],downcast="float")
    
#COMPANY WEIGHTAGE CALCULATION
empty_3 = []
sum_df = portfolio_data.sum(axis=1)
total_weightage = sum_df ["Position(BDT)"]
for company in full_portfolio:
    x = (portfolio_data[company]["Position(BDT)"])/weightage
    empty_3.append(x)
company_weight = dict(zip(full_portfolio,empty_3))
c_weight = pd.DataFrame(company_weight,index=["Company Weight"]) 
total_data = portfolio_data.append(c_weight)
total_data 

#PORTFOLIO BETA COLLECTION
p_beta = 1
for company in full_portfolio:
    weight = total_data[company]["Company Weight"]
    beta = total_data[company]["Beta"]
    mlt_beta = (weight*beta)
    p_beta*=mlt_beta
print(p_beta)

In [ ]:
data = {}
for company in full_portfolio:
    driver = webdriver.Chrome("C:/Projects/chromedriver_win32/chromedriver.exe")
    driver.get('https://www.amarstock.com/stock/%s'% company)
    time.sleep(1)
    pe_ratio = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[1]/div/div[3]") 
    beta = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[5]/div/div[3]")
    eps = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[2]/div/div[3]")
    price_to_book = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[3]/div/div[3]")
    div_yield = driver.find_element_by_xpath("/html/body/section[1]/div/div[2]/div[6]/div/div[3]")
    ratios = [pe_ratio.text,beta.text,eps.text,price_to_book.text,div_yield.text]
    data[company] = ratios
    driver.quit()
df = pd.DataFrame(data, index = ["P/E Ratio","Beta","EPS","Price-to-book","Dividend Yield",])
portfolio_data = df.append(df_1)
for company in full_portfolio:
    portfolio_data[company] = pd.to_numeric(portfolio_data[company],downcast="float")
    
#COMPANY WEIGHTAGE CALCULATION
empty_3 = []
sum_df = portfolio_data.sum(axis=1)
total_weightage = sum_df ["Position(BDT)"]
for company in full_portfolio:
    x = (portfolio_data[company]["Position(BDT)"])/weightage
    empty_3.append(x)
company_weight = dict(zip(full_portfolio,empty_3))
c_weight = pd.DataFrame(company_weight,index=["Company Weight"]) 
total_data = portfolio_data.append(c_weight)
total_data 

In [ ]:
#SCRAPING PORTFOLIO RELATED DATA
print ("Fetching portfolio relevant publicly available news from the past 7 days.......\n")
counter = 0
driver = webdriver.Chrome("C:/Projects/chromedriver_win32/chromedriver.exe")
driver.get('https://main.dsebd.org/news_archive_7days.php')
scrape_ticker = driver.find_elements_by_css_selector('tr')
for news in scrape_ticker:
    if (counter == 2):
                print(news.text)
                counter +=1
    elif(counter == 3):
        print(news.text)
        print("______________________________________________________________________________")
        counter = 0
    else:
        for ticker in full_portfolio:
            if (ticker in news.text):
                print (news.text)
                counter+=1
driver.quit()                
print("\nFinished fetching relevant news")